# Аналитика данных

In [ ]:
train_aspects = pd.read_csv('train_aspects.txt', sep='\t', names=['id', 'category', 'word', 'start', 'end', 'sentiment'])
train_aspects

,id,category,word,start,end,sentiment
0,3976,Whole,ресторане,71,80,neutral
1,3976,Whole,ресторанах,198,208,neutral
2,3976,Whole,ресторане,256,265,neutral
3,3976,Service,Столик бронировали,267,285,neutral
4,3976,Service,администратор,322,335,positive
...,...,...,...,...,...,...
4758,16630,Service,обслуживание,85,97,positive
4759,16630,Food,Еда,99,102,positive
4760,16630,Service,персоналу,244,253,positive
4761,16630,Whole,ресторан,294,302,positive


## Поиск сущностей с единичной встречаемостью

In [ ]:
from collections import Counter

In [ ]:
def find_single(df, category):

  cat = df.loc[df['category'] == category]
  cat_ = list(cat['lemmas'])
  d = Counter(cat_).most_common()

  single = []
  for w in d:
    if w[1] == 1:
      single.append(w[0])

  ids = []
  for w in single:
    id = cat[cat['lemmas'] == w]['id'].values
    if len(id) == 1:
      ids.append(id[0])

  result = pd.DataFrame(columns=['id', 'category', 'lemmas', 'count'])
  result['id'] = ids
  result['category'] = category
  result['lemmas'] = single
  result['count'] = 1

  return result

In [ ]:
categories = ['Whole', 'Service', 'Interior', 'Price', 'Food']

In [ ]:
for c in categories:
  res = find_single(train_aspects)
  res.to_csv(f'single{c}.csv', sep='\t', index=False)

In [ ]:
# Но вот в счет их ВКЛЮЧИТЬ они не забыли
reviews[reviews.idx==34307]

,idx,text
257,34307,"Недавно пришли компанией, давно не были в этом заведении и лучше бы не приходили!!!! Была пятница, людей в зале под завязку, сели за единственный свободный стол, так столы стоят настолько близко, что выйти из- за него невозможно((( но это не самое страшное...... Очень долго несут заказ либо вобще про тебя забывают, все время приходят и переспрашивают( ну если у тебя плохая память- возьми бумагу и записывай) . Мы брали горячее, салатики, пиво и закуски к пиву, которые так и не дождались, хотя по прошествии часа 1 раз даже напомнили, чего мы делать не должны в принципе. Но вот в счет их ВКЛЮЧИТЬ они не забыли!!!! Салаты раньше были вкуснее. Еду приносят черти как, во- первых без приборов, все время нужно напоминать, во вторых, по времени ужасно,1 из нас уже час как поел, а второму еще даже готовить видимо не начали.Официанта то не дождешься, то еще доесть не успел, уже тарелку забирают. В зале стоит галдеж, музыку вобще не слышно. В это заведение мы раньше ходили с удовольствием, что случилось- непонятно. Больше ни ногой!!!! Цены пивного ресторана, а обслуживание дешевой привокзальной пивнушки((((((("


In [ ]:
aspects[(aspects.idx==34307) & (aspects.aspect == 'счет')]

,idx,category,aspect,start,end,mood
4260,34307,Service,счет,584,588,negative


In [ ]:
# 2 раз не принесли салат, но в счет включили
reviews[reviews.idx==34710]

,idx,text
192,34710,"Удивляюсь отзывам про хорошее обслуживание. Были 4 раза из них только 1 раз нормально обслужили: 1 раз не дали меню, очень долго несли напитки; 2 раз не принесли салат, но в счет включили; 3 раз вообще эпик фейл - хамское и небрежное отношение официантов... Меню на столе нет при этом официантка подходит и спрашивает: ""Что вы выбрали?"" Из чего? После этого нам дали меню ланча (обычное меню видимо только одно на весь ресторан). После этого официантка удалилась и больше к столу не подходила прождав минут 10, позвали другую. Через 3 мин приняла заказ. Вместо заказанного супа принесла другой, через 2 мин. поменяла. Напитки принесла только после десерта (после просьбы принести напитки!!!!). Ужасно одним словом! Да кухня отличная, кеталонский крем чудо. Но где берут таких официантов??? Честно, неприятное впечатление, думала все изменилось, ан нет=("


In [ ]:
aspects[(aspects.idx==34710) & (aspects.aspect == 'счет')]

,idx,category,aspect,start,end,mood
3150,34710,Price,счет,174,178,negative


In [ ]:
# счет несли довольно долго
reviews[reviews.idx==31071]

,idx,text
202,31071,"Нормальное заведение, но на мой взгляд, не ""великолепное"". Во-первых, очень шумно: зал небольшой и когда все разом разговаривают (а посетителей было много), получается такой гул, что не слышно, о чем говорят твои спутники, сидящие напротив и сами столы расположены довольно близко друг к другу. Во-вторых, довольно-таки дорого. В-третьих, обслуживание: было очень неприятно топтаться в центре зала в верхней одежде (все официанты бегали по залу, никто не встретил и не показал где гардероб - он оказался шкафом между туалетами:)), официантка приносила сначала блюда, а потом уже приборы, счет несли довольно долго, когда уходили, девушка из гардероба выдала не все куртки и ушла... мелочи, но из них складывается впечатление. Интерьер и месторасположение ресторана понравились. Кухня неплохая, никаких претензий, попробовала равиоли с трюфелями и рикоттой (380 р.) и крем-суп из тыквы с пармезаном (380 р.) (так получилось, что выбрала самые недорогие блюда из меню:)). Вино красное ""домашнее"" - 260 рублей/бокал. Чек на 4 девушек с вином получился около 6000."


In [ ]:
aspects[(aspects.idx==31071) & (aspects.aspect == 'счет')]

,idx,category,aspect,start,end,mood
3338,31071,Price,счет,588,592,neutral


Кажется, что всё это должно относиться тогда к категории Service, так как оценивается формат работы сотрудников с счётом. Но в двух случаях из трёх это относится к цене.

Ниже есть пример, в котором текст "белого сухого вина", кажется, должно относиться к еде, но размечено как сервис. Это можно было предположить, если белое сухое вино предлагали как комплимент от ресторана, но в предложении написано "заказали".

In [ ]:
reviews[reviews.idx==14539]

,idx,text
259,14539,"27 апреля гуляли с женой по Пушкину и случайно зашли перекусить. Заказали шашлыки из свинины, овощи на гриле, картофель ""Айдахо"" по домашнему морсу и по бокалу французского белого сухого вина. Что понравилось. Очень вкусный, большой шашлык на шампуре с оригинальной подставкой. Когда его вносят- горит. Овощи на гриле и ""Айдахо"" тоже очень хороши. Одним словом кухня отличная. Интерьер ""Ресторанчика"" на меня особого впечатления не произвел. За счет того что стены из стеклопакетов ощущение что находишься в теплице. Но в этом тоже что-то есть. Обслуживание мне понравилось, но было одно но: При заказе официантка предложила сама взять морс по 500 млт, а принесли высокие тонкие стаканы по 200 млт. Я сразу не обратил внимания. А в конце когда сказал об этом официантке она очень смутилась. Думаю- не заостри я на этом внимания, в счете была бы сумма за два по 500 млт, а не за два по 200 млт. Впрочем это только мои догадки. Обслуживали нас девушка и молодой человек. Быстро, вежливо , аккуратно. Ждать заказа долго не пришлось. Убирали использованную посуду быстро. Все доброжелательно и с улыбкой. Счет принесли на 2050 руб. На мой взгляд очень не плохое заведение. Рекомендую."


In [ ]:
aspects[(aspects.idx==14539) & (aspects.aspect == 'белого сухого вина')]

,idx,category,aspect,start,end,mood
4290,14539,Service,белого сухого вина,173,191,neutral
